In [ ]:


import numpy as np 
import pandas as pd 


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:

!pip uninstall -y cudf-cu12 pylibcudf-cu12 cudf-polars-cu12 libcugraph-cu12 pylibcugraph-cu12 libraft-cu12 pylibraft-cu12 rmm-cu12 google-cloud-bigquery google-cloud-bigquery-storage pandas-gbq bigframes rich

!pip install -q pytube pydub openai-whisper keybert sentencepiece torch torchvision torchaudio
!pip install -q "transformers>=4.35.0" "pyarrow>=19.0.0,<20.0.0" "pydantic<2.12"


Found existing installation: cudf-cu12 25.2.2
Uninstalling cudf-cu12-25.2.2:
  Successfully uninstalled cudf-cu12-25.2.2
Found existing installation: pylibcudf-cu12 25.2.2
Uninstalling pylibcudf-cu12-25.2.2:
  Successfully uninstalled pylibcudf-cu12-25.2.2
Found existing installation: cudf-polars-cu12 25.6.0
Uninstalling cudf-polars-cu12-25.6.0:
  Successfully uninstalled cudf-polars-cu12-25.6.0
Found existing installation: libcugraph-cu12 25.6.0
Uninstalling libcugraph-cu12-25.6.0:
  Successfully uninstalled libcugraph-cu12-25.6.0
Found existing installation: pylibcugraph-cu12 25.6.0
Uninstalling pylibcugraph-cu12-25.6.0:
  Successfully uninstalled pylibcugraph-cu12-25.6.0
Found existing installation: libraft-cu12 25.2.0
Uninstalling libraft-cu12-25.2.0:
  Successfully uninstalled libraft-cu12-25.2.0
Found existing installation: pylibraft-cu12 25.2.0
Uninstalling pylibraft-cu12-25.2.0:
  Successfully uninstalled pylibraft-cu12-25.2.0
Found existing installation: rmm-cu12 25.2.0
Uninst

In [4]:
from pytube import YouTube
from pydub import AudioSegment
import whisper
from transformers import pipeline
from keybert import KeyBERT
import torch

2025-11-01 09:50:41.720780: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761990641.886904      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761990641.938000      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
!pip uninstall -y pytube pytube3


Found existing installation: pytube 15.0.0
Uninstalling pytube-15.0.0:
  Successfully uninstalled pytube-15.0.0


In [9]:
!pip install -q git+https://github.com/pytube/pytube


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [12]:
!pip install -q yt_dlp pydub

from pydub import AudioSegment
import yt_dlp
import os

def download_audio_ytdlp(url, output_path="lecture_audio.mp3"):
    temp_file = "temp_audio"
    ydl_opts = {
        "format": "bestaudio/best",
        "outtmpl": f"{temp_file}.%(ext)s",
        "quiet": True,
        "postprocessors": [{"key": "FFmpegExtractAudio", "preferredcodec": "mp3"}]
    }
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])

        # Find downloaded mp3 file
        for file in os.listdir():
            if file.startswith(temp_file) and file.endswith(".mp3"):
                os.rename(file, output_path)
                break
        return output_path
    except Exception as e:
        print("❌ Error downloading audio:", e)
        return None

# ✅ Example
youtube_link = "https://youtu.be/UWYT7KitauA"
audio_path = download_audio_ytdlp(youtube_link)
print("✅ Audio downloaded:", audio_path)


         player = https://www.youtube.com/s/player/c6d7bdc9/player_ias.vflset/en_US/base.js
         n = zuqJ98px2XXRWuCR ; player = https://www.youtube.com/s/player/c6d7bdc9/player_ias.vflset/en_US/base.js
         Please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U


✅ Audio downloaded: lecture_audio.mp3                      


In [ ]:
!pip install -q openai-whisper

import whisper

def transcribe_audio(audio_path):
    print("🎧 Loading Whisper model...")
    model = whisper.load_model("tiny")  
    print("🔊 Transcribing audio...")
    result = model.transcribe(audio_path)
    
    transcript = result["text"]
    print("\n✅ Transcription complete!\n")
    return transcript

# ✅ Example
transcript = transcribe_audio("lecture_audio.mp3")

with open("lecture_transcript.txt", "w") as f:
    f.write(transcript)

print("📝 Transcript saved to lecture_transcript.txt")


🎧 Loading Whisper model...


100%|██████████████████████████████████████| 72.1M/72.1M [00:00<00:00, 206MiB/s]


🔊 Transcribing audio...

✅ Transcription complete!

📝 Transcript saved to lecture_transcript.txt


In [16]:
!pip install -q transformers sentencepiece torch keybert


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [15]:
from transformers import pipeline
import re

# ====== LOAD SUMMARIZER MODEL ======
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# ====== LOAD QUESTION GENERATOR ======
qg = pipeline("text2text-generation", model="iarfmoose/t5-base-question-generator")

# ====== LOAD KEYWORD EXTRACTOR (using regex fallback) ======
def extract_keywords(text, n=10):
    words = re.findall(r'\b[A-Za-z]{5,}\b', text)
    freq = {}
    for w in words:
        freq[w.lower()] = freq.get(w.lower(), 0) + 1
    sorted_words = sorted(freq.items(), key=lambda x: x[1], reverse=True)
    return [w for w, _ in sorted_words[:n]]

# ====== MAIN FUNCTION ======
def analyze_lecture(transcript, chunk_size=1000):
    print("📖 Summarizing lecture...")
    chunks = [transcript[i:i+chunk_size] for i in range(0, len(transcript), chunk_size)]
    summaries = [summarizer(chunk, max_length=150, min_length=60, do_sample=False)[0]['summary_text']
                 for chunk in chunks]
    full_summary = " ".join(summaries)

    print("🧠 Generating questions...")
    q_output = qg(full_summary + " generate 5 questions about this text")[0]['generated_text']

    print("🔍 Extracting key topics...")
    keywords = extract_keywords(full_summary)

    return full_summary, q_output, keywords

# ====== RUN PIPELINE ======
with open("lecture_transcript.txt") as f:
    transcript = f.read()

summary, questions, keywords = analyze_lecture(transcript)

print("\n✅ Summary:\n", summary)
print("\n❓ Quiz Questions:\n", questions)
print("\n🧩 Key Topics:\n", keywords)


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Device set to use cuda:0


📖 Summarizing lecture...


Your max_length is set to 150, but your input_length is only 26. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=13)
Token indices sequence length is longer than the specified maximum sequence length for this model (683 > 512). Running this sequence through the model will result in indexing errors


🧠 Generating questions...
🔍 Extracting key topics...

✅ Summary:
 This chapter talks about democracy and how power is shared among the government branches. In this lesson we study the political situations of Belgium and Sri Lanka to understand why Power Sharing is important. We also learn about different ways of power sharing. First, let us see the situation in Belgium. In the capital city Brussels, 80% of people speak French and only 20% speak Dutch. Sri Lanka is an island nation near the southern coast of Tamil Nadu India. Its population is about 2 crore which is similar to Ariana. There are 2 major ethnic groups in Sri Lanka. Sinhala speakers are about 74% the majority and Tamil speakers are 18% which are minority. Sri Lanka became independent in 1948. Sinhala leaders being the majority wanted to dominate the government. In 1956, an act was possibl in 1956 to create a state of emergency. Most Tamil's are Hindus or Muslims. About 7% are Christians from bothSinhala and Tamil groups.  

In [18]:
from transformers import pipeline
from keybert import KeyBERT
import re

# Load models
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
qa = pipeline("question-answering", model="deepset/roberta-base-squad2")
kw_model = KeyBERT()

# Auto-length summarizer
def summarize_text(text, mode="short"):
    length = len(text.split())
    if length < 50:
        return text
    if mode == "short":
        max_len, min_len = 80, 30
    elif mode == "medium":
        max_len, min_len = 150, 60
    else:
        max_len, min_len = 250, 100

    chunks = [text[i:i+1000] for i in range(0, len(text), 1000)]
    summaries = summarizer(chunks, max_length=max_len, min_length=min_len, do_sample=False)
    return " ".join([s["summary_text"] for s in summaries])

# Keyword extractor
def extract_keywords(text, n=8):
    try:
        return [kw for kw, _ in kw_model.extract_keywords(text, top_n=n)]
    except:
        words = re.findall(r'\b[A-Za-z]{5,}\b', text)
        freq = {}
        for w in words:
            freq[w.lower()] = freq.get(w.lower(), 0) + 1
        return [w for w, _ in sorted(freq.items(), key=lambda x: x[1], reverse=True)[:n]]

# Interactive mode
def interactive_summary(transcript):
    print("🎧 Lecture Summary Modes:")
    print("1️⃣ Short  2️⃣ Medium  3️⃣ Detailed")
    mode = {"1": "short", "2": "medium", "3": "long"}.get(input("👉 Choose summary length (1/2/3): ").strip(), "short")

    summary = summarize_text(transcript, mode)
    print("\n📄 Summary:\n", summary)

    keywords = extract_keywords(summary)
    print("\n🧩 Keywords:", ", ".join(keywords))

    print("\n💬 Ask anything about this lecture! (type 'exit' to stop)")
    while True:
        q = input("❓ You: ")
        if q.lower() in ["exit", "quit"]:
            print("👋 Ending session.")
            break
        ans = qa(question=q, context=summary)
        print("🤖 AI:", ans["answer"])

# Run
with open("lecture_transcript.txt") as f:
    transcript = f.read()

interactive_summary(transcript)


Device set to use cuda:0


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

🤖 AI: Thousands of people from both sides


❓ You:  which both


🤖 AI: Belgium and Sri Lanka


❓ You:  exit


👋 Ending session.
